# Packages 

In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
import os
import base64
import re
import pickle
import dash
import dash_bootstrap_components as dbc

from dash.exceptions import PreventUpdate
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Output
from dash.dependencies import Input
from plotly.subplots import make_subplots
from textblob import TextBlob

from emotions_model import count_tokens
from emotions_model import apply_features
from emotions_model import count_n_char

C:\Anaconda3\lib\site-packages\dash_bootstrap_components\_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


# Images and Model


In [2]:
# Relative paths
cwd=os.getcwd()

fpath='.\images\initial.jpg'

test_base64 = base64.b64encode(open(fpath, 'rb').read()).decode('ascii')


# Deserializing model object
handler = open('model.pkl', "rb")

pipeML= pickle.load(handler)




# App layout and controls

In [3]:
app=dash.Dash(__name__,external_stylesheets=[dbc.themes.SPACELAB,
                                             "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css"])



app.layout=html.Div([
    
#Colors and app title
    dbc.Row([dbc.Col(
                    html.H1('Anger',style={"color":"black","background-color":"#F13027",
                                          "text-align":"center"})
                    ,width={'size':2,'offset':0,'order':1}
                    ),
            dbc.Col(
                    html.H1("Love",style={"color":"black","background-color":"#F127A1",
                                          "text-align":"center"})
                    ,width={'size':2,'order':2}
                    ),
            dbc.Col(
                    html.H1("Joy",style={"color":"black","background-color":"#FFF633",
                                        "text-align":"center"})
                    ,width={'size':2,'order':3}
                    ), 
            dbc.Col(
                    html.H1("Surprise",style={"color":"black","background-color":"#27F143",
                                             "text-align":"center"})
                    ,width={'size':2,'order':4}
                    ), 
            dbc.Col(
                    html.H1("Sadness",style={"color":"black","background-color":"#279BF1",
                                            "text-align":"center"})
                    ,width={'size':2,'order':5}
                    ), 
            dbc.Col(
                    html.H1("Fear",style={"color":"black","background-color":"#9827F1",
                                         "text-align":"center"})
                    ,width={'size':2,'order':6}
                    ), 
            ]),

    
#Free text input for user text.  The dcc store is used to store data in browser and
    #can share data between callbacks 
    dbc.Row([dbc.Col([
                    html.Div(["How are you feeling"],style={'font-size':24}),
                    dcc.Input(id='text_input',placeholder='Enter text..',type='text',style={'font-size':24,
                                                                                      'width':'80%'})],
            width={'size':6,'offset':1}),
            ]),
    
    dcc.Download(id='download_excel'),    
    dcc.Store(id='saved_data',data={}),
    dcc.Store(id='df_filtered',data={}),

# Variable image
    dbc.Row([

        dbc.Col([
                html.Br(),
                html.Img(id='picture',src='data:image/png;base64,{}'.format(test_base64)),

                ],
                width={'offset':5,'order':1},

                ),#End dbc.Col
        
            ]), #End dbc.Row
        

    dbc.Row([html.Br()]),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='sentiment')],width={'size':4,'offset':4}),
        ]), #End dbc.Row
    dbc.Row([
        dbc.Col([
            html.Table([
                html.Tr([html.Td(['Sentiment Subjectivity:  '],
                                 style={'font-size':24}), 
                         html.Td(id='subj',
                                 style={'text-align':'right','font-size':24})],
                         style={'text-align':'right','font-size':24}),
                html.Tr([html.Td('The subjectivity is in the range [0.0, 1.0]',
                                  style={'text-align':'right','font-size':16})],
                        style={'text-align':'right','font-size':16}),
                html.Tr([html.Td('where 0.0 is very objective and 1.0 is very subjective',
                                  style={'text-align':'right','font-size':16})],
                        style={'text-align':'right','font-size':16}),
                        ]), # End Table 
                ],width={'offset':5}) # End Col

    ]) #End Row


        
        

                    
]) #End html.Div

        

# Callbacks


In [4]:
# 1) Populate the graph with sentiment and subjectivity
# 2) Populate the image based on the value of model output 

@app.callback(
    Output(component_id='sentiment',component_property='figure'),
    Output('picture', 'src'),
    Output('subj','children'),
    Input(component_id='text_input',component_property='value')

)

def set_graph(text):
    
    print(text)
    '''
    INPUT
    text - a string of text based on the user input in the application

    OUTPUT
    fig - a figure to show the sentiment and subjectivity of the text using the TextBlob library
    '''    
    
    
    def SetColor(y):
        '''
        INPUT
        y - the y-value for the graph

        OUTPUT
        col - color based on y-value
        '''            
        if y>=0:
            col= "#3F9C35"
        else:
            col= "red"
        return col

    
    
    if text is None:
        raise PreventUpdate
    
    
    #Sentiment from Textblob

    tb_phrase = TextBlob(text)
    
    phrase_corrected=str(tb_phrase.correct())
    
    
    if tb_phrase==phrase_corrected:
#         print('No spelling errors')
        pass
    else:
#         print('Spelling error! Fixed')
#         print(phrase_corrected)
        tb_phrase=TextBlob(phrase_corrected)
    

    sentiment=tb_phrase.sentiment

    polarity=sentiment.polarity

    subjectivity=sentiment.subjectivity

    #Variables for graph
    x=['Sentiment']

    ydf=pd.DataFrame([polarity],columns=["y1"])

    if polarity<0:

        y_text = ['Negative']
    else:
        y_text = ['Positive']


    #Graph properties
    fig = go.Figure(data=go.Bar(x=x, y=ydf['y1'],text=y_text,
                                marker=dict(color = list(map(SetColor, ydf["y1"])))))

    fig.update_traces(marker_line_color='black',marker_line_width=1.5,opacity=0.6)


    fig.update_layout(title_text="Sentiment score: -1 to 1",
                      title_font_size=30,
                      font=dict(size=24),
                      title_x=0.5)

    fig.update_yaxes(range=[-1,1],tickvals=[-1, -0.5, 0, 0.5,1],
                     zeroline=True,zerolinewidth=2,
                     zerolinecolor='black',showgrid=True)  
    
    #Instantiate feature generation object and use methods for preprocessing

    numeric_cols=['n_tokens','n_i']

    str_cols='document'

    features_created=['n_characters']
    
    
    
    
    new_text_df=pd.DataFrame({'document':[text]},columns=['document'])
    
    new_text_df=apply_features(new_text_df)
    
    print(new_text_df.head())
    
    # Use trained model on new text and output result via image

    pred=pipeML.predict(new_text_df)
    
    pred_emotion=list(pred)[0]
    
    
    fpath=f".\images\{pred_emotion}.jpg"
    
    test_base64 = base64.b64encode(open(fpath, 'rb').read()).decode('ascii')
    src='data:image/png;base64,{}'.format(test_base64)
    
    return fig,src,round(subjectivity,1)

In [ ]:
if __name__=='__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/May/2022 17:13:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 17:13:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 17:13:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 17:13:25] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 17:13:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 17:13:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 17:13:25] "POST /_dash-update-component HTTP/1.1" 204 -


None
i
i 


127.0.0.1 - - [18/May/2022 17:13:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2022 17:13:27] "POST /_dash-update-component HTTP/1.1" 200 -


Generating featuresGenerating features

  document  n_tokens  n_i  sentiment
0       i          1    1        0.0  document  n_tokens  n_i  sentiment
0        i         1    0        0.0

i a
Generating features

...

I felt it was romanti
Generating features
                document  n_tokens  n_i  sentiment
0  I felt it was romanti         5    1        0.0
I felt it was romantic
Generating features
                 document  n_tokens  n_i  sentiment
0  I felt it was romantic         5    1        0.0


127.0.0.1 - - [18/May/2022 17:21:05] "POST /_dash-update-component HTTP/1.1" 200 -


I felt it was romantic 
Generating features
                  document  n_tokens  n_i  sentiment
0  I felt it was romantic          5    1        0.0


127.0.0.1 - - [18/May/2022 17:21:06] "POST /_dash-update-component HTTP/1.1" 200 -



Generating features
  document  n_tokens  n_i  sentiment
0                  0    0        0.0
